In [14]:
import inspect
from functools import wraps

In [15]:
class M(type):

    def __new__(mcl, name, bases, nmspc):
        print("in meta")
        return super(M, mcl).__new__(mcl, name, bases, nmspc)

In [18]:
def singleton_on_features(*features):
    print("start decoration")
    instances = {}  # { <features_dict_frozenset>: <instance> }

    def singleton(cls):
        print("start singleton")
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

        func = cls.__init__

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                return func(self, *args, **kwargs)
#             else:
#                 raise Exception("Conflict naming: ___init__called")

        def _new(clsobj, *args, **kwargs):
            """
            使用元编程实现单例
            """
            fts = {k: v for k, v in kwargs.items() if k in features}
            fts_set = set(fts.items())
            fts_frozenset = frozenset(fts.items())

            instanced = fts_frozenset in instances
            if not instanced or (instanced and len(fts_set - set(kwargs.items())) == len(fts_set)):  # 说明没有任何交集
                instances[fts_frozenset] = object.__new__(clsobj)  # 则实例化
            return instances[fts_frozenset]  # 否则返回单例

        # 拦截__new__方法
        setattr(cls, "__new__", classmethod(_new))
        # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
        # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
        # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
        # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
        setattr(cls, "__init__", _wrapper_init)
        print("stop singleton")
        return cls
    print("stop decoration")
    return singleton

In [19]:
@singleton_on_features("sn")
class A(metaclass=M):
    def __init__(self, sn):
        print("A: init")
        self.sn = sn

start decoration
stop decoration
in meta
start singleton
stop singleton


## 结论

先装饰，再元类，再类，再实例化